Бустинг — это мощный ансамблевый метод машинного обучения, который комбинирует множество слабых моделей (обычно неглубоких деревьев решений) в одну сильную модель. Ключевая идея: последовательное обучение, где каждая новая модель исправляет ошибки предыдущих

 Вот как это работает:

1. Последовательное обучение

  В отличие от случайного леса, где деревья строятся параллельно, бустинг создает деревья одно за другим. Каждое новое дерево фокусируется на ошибках, которые сделали предыдущие.

2. Коррекция ошибок

  После добавления каждого дерева модель пересматривает:
  Какие объекты она плохо предсказывает
  Какие признаки наиболее важны для исправления ошибок

3. Взвешенное голосование

  Каждое дерево в ансамбле получает "вес" в зависимости от его точности. Более точные деревья имеют больший вес при итоговом предсказании.

4. Контроль переобучения

  Используются три основных механизма:

  - Ограничение глубины деревьев (обычно 3-6 уровней)
  - Малый шаг обучения (learning rate 0.01-0.3)
  - Случайные подвыборки данных (subsampling)




In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification

# Создаем искусственные данные
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Инициализируем модель
boost = GradientBoostingClassifier(
    n_estimators=100,     # Количество деревьев
    learning_rate=0.1,    # Размер шага
    max_depth=3,          # Глубина каждого дерева
    random_state=42
)

# Обучение модели
boost.fit(X, y)

# Оценка точности
print(f"Точность: {boost.score(X, y):.3f}")

Точность: 0.981


Меняйте число деревьев — увидите, как ансамбль становится точнее

Пробуйте разные скорости обучения (learning rate)

Сравните глубину деревьев:

Малая глубина (1-3) — простые модели, устойчивые к шуму

Большая глубина — риск переобучения

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from ipywidgets import interact, widgets

# Генерация данных (как в вашем примере)
X, y = make_regression(n_features=1, n_samples=100, noise=20, random_state=42)
X = X * 50 + 50  # Масштабируем к диапазону площадей [0-100 м²]
y = y * 20000 + 100000  # Масштабируем к ценам [50k-150k руб]

def interactive_adaboost(n_estimators=50, learning_rate=0.1, max_depth=3):
    # Убедимся, что X имеет правильную форму (n_samples, n_features)
    X_reshaped = X.reshape(-1, 1) if X.ndim == 1 else X

    model = AdaBoostRegressor(
        estimator=DecisionTreeRegressor(max_depth=max_depth),
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=42
    )
    model.fit(X_reshaped, y)

    plt.figure(figsize=(12, 6))

    x_test = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
    y_pred = model.predict(x_test)

    print(f"R² score: {model.score(X_reshaped, y):.3f}")
    print(f"Пример прогноза для 50 м²: {model.predict([[50]])[0]:.0f} руб")


    plt.scatter(X, y, alpha=0.7, label='Данные')
    plt.plot(x_test, y_pred, 'r-', linewidth=2, label='Предсказание')
    plt.xlabel('Площадь (м²)')
    plt.ylabel('Цена (руб)')
    plt.title(f'AdaBoost (Деревьев: {n_estimators}, Глубина: {max_depth})')
    plt.legend()
    plt.grid(True)

    plt.show()


interact(interactive_adaboost,
         n_estimators=widgets.IntSlider(value=50, min=10, max=200, step=10, description='Число деревьев:'),
         learning_rate=widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Learning rate:'),
         max_depth=widgets.IntSlider(value=3, min=1, max=10, description='Глубина деревьев:'));

interactive(children=(IntSlider(value=50, description='Число деревьев:', max=200, min=10, step=10), FloatSlide…

В отличие от AdaBoost, Gradient Boosting:

На каждом шаге вычисляет градиент функции потерь

Строит дерево, которое предсказывает антиградиент (направление исправления ошибок)

Делает шаг в этом направлении (с учетом learning rate)

Куда "бустится" градиент?
Каждое новое дерево приближает не сами значения, а остатки (разницу между текущими предсказаниями и истинными значениями):

Интерактивное исследование:



In [39]:
from sklearn.ensemble import GradientBoostingRegressor

def interactive_gradient_boosting(n_estimators=100, learning_rate=0.1, max_depth=3):
    X_reshaped = X.reshape(-1, 1) if X.ndim == 1 else X

    model = GradientBoostingRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=42
    )
    model.fit(X_reshaped, y)

    plt.figure(figsize=(12, 6))

    x_test = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
    y_pred = model.predict(x_test)


    print(f"R² score: {model.score(X_reshaped, y):.3f}")
    print(f"Пример прогноза для 75 м²: {model.predict([[75]])[0]:.0f} руб")

    plt.scatter(X, y, alpha=0.7, label='Данные')
    plt.plot(x_test, y_pred, 'r-', linewidth=2, label='Предсказание')
    plt.xlabel('Площадь (м²)')
    plt.ylabel('Цена (руб)')
    plt.title(f'Gradient Boosting (Деревьев: {n_estimators})')
    plt.legend()
    plt.grid(True)

    plt.show()

interact(interactive_gradient_boosting,
         n_estimators=widgets.IntSlider(value=100, min=10, max=500, step=10, description='Число деревьев:'),
         learning_rate=widgets.FloatSlider(value=0.01, min=0.0005, max=1.0, step=0.0005, description='Learning rate:'),
         max_depth=widgets.IntSlider(value=3, min=1, max=10, description='Глубина деревьев:'));

interactive(children=(IntSlider(value=100, description='Число деревьев:', max=500, min=10, step=10), FloatSlid…